# 6.2.1 -- Listing 1 Index two docs

In [2]:
import requests
import json

# PUT albinoelephant/docs/1
# { "title":"albino", "body": "elephant"}
# PUT albinoelephant/docs/1
# { "title":"elephant", "body": "elephant"}


headers = {'Content-type': 'application/json', 'Accept': 'application/json'}    

resp = requests.put('http://localhost:9200/albinoelephant/docs/1',
                    data=json.dumps({'title': 'albino',
                                     'body': 'elephant'}),headers=headers)
resp = requests.put('http://localhost:9200/albinoelephant/docs/2',
                    data=json.dumps({'title': 'elephant',
                                     'body': 'elephant'}),headers=headers)
print resp.text

{"_index":"albinoelephant","_type":"docs","_id":"2","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":1,"_primary_term":1}


# 6.2.1, Listing 2 Searching for the infamous ‘albino elephant’

In [4]:
query = {
    'query': {
        'multi_match': {
            'query': 'albino elephant',
            'type': 'most_fields',
            'fields': ['title', 'body']
        }
    }
}

# Notice how "title:elephant body:elephant gets same score as title:albino body:elephant
# This 'most fields' is equivelant to Lucene's MultiFieldQueryParser 
# https://lucene.apache.org/core/4_1_0/queryparser/org/apache/lucene/queryparser/classic/MultiFieldQueryParser.html
# So Chuck complained and proposed a fix in 
# https://issues.apache.org/jira/browse/LUCENE-323
#GET albinoelephant/docs/_search?pretty=true #C
#{
#    'query': {
#        'multi_match': {
#            'query': 'albino elephant',
#            'type': 'most_fields',
#            'fields': ['title', 'body']
#        }}}
#
#
#
#
resp = requests.get('http://localhost:9200/albinoelephant/docs/_search?pretty=true', data=json.dumps(query),headers=headers)
print resp.status_code
print resp.text

200
{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 2,
      "relation" : "eq"
    },
    "max_score" : 0.8754687,
    "hits" : [
      {
        "_index" : "albinoelephant",
        "_type" : "docs",
        "_id" : "1",
        "_score" : 0.8754687,
        "_source" : {
          "body" : "elephant",
          "title" : "albino"
        }
      },
      {
        "_index" : "albinoelephant",
        "_type" : "docs",
        "_id" : "2",
        "_score" : 0.8754687,
        "_source" : {
          "body" : "elephant",
          "title" : "elephant"
        }
      }
    ]
  }
}



In [9]:

query = {
    'query': {
        'multi_match': { 
            'query': 'albino OR elephant',  #User's query
            'fields': ['title', 'body'],      
            'type' : 'cross_fields'
         }
    },
    'size': 5
}

resp = requests.get('http://localhost:9200/albinoelephant/docs/_search?pretty=true', data=json.dumps(query),headers=headers)
print resp.status_code
print resp.text

200
{
  "took" : 0,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 2,
      "relation" : "eq"
    },
    "max_score" : 0.8754687,
    "hits" : [
      {
        "_index" : "albinoelephant",
        "_type" : "docs",
        "_id" : "1",
        "_score" : 0.8754687,
        "_source" : {
          "body" : "elephant",
          "title" : "albino"
        }
      },
      {
        "_index" : "albinoelephant",
        "_type" : "docs",
        "_id" : "2",
        "_score" : 0.18232156,
        "_source" : {
          "body" : "elephant",
          "title" : "elephant"
        }
      }
    ]
  }
}

